In [ ]:
import xarray as xr
import time
import itertools
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random
import seaborn as sns

from py_wake.deficit_models import EddyViscosityModel

import sys

sys.path.append("..")

import src.utils as utils
import src.pywake_utils as py_wake_utils
from src.data_utils import load_netcfd
import src.data_utils as data_utils

# Discretization parameters tuning

In [ ]:
DEFICIT_THRESHOLD = 1 / 100  # putting it to 1/1000, the region becomes too big

X_START_FACTOR = 2
X_END_FACTOR = 50
Y_START_FACTOR = -10
Y_END_FACTOR = 10
GRID_STEP_FACTOR = 1 / 8

TURBINE_DIAMETER = 198
TURBINE_HUB_HEIGHT = 119
TURBINE_POWER_NORM = 10000
TURBINE_X = [0]
TURBINE_Y = [0]

WS_RANGE = range(10, 26)
WIND_DIRECTION = 270
TI_STEP = 0.01
CT_STEP = 0.01
TIs = utils.my_arange(0, 1, TI_STEP)  # the ti is percentage
CTs = utils.my_arange(0.1, 24 / 25, CT_STEP)


horizontal_grid = py_wake_utils.get_discretized_grid(
    TURBINE_DIAMETER,
    X_START_FACTOR,
    X_END_FACTOR,
    Y_START_FACTOR,
    Y_END_FACTOR,
    GRID_STEP_FACTOR,
)

x_ranges = []
y_ranges = []

for wind_speed in random.sample(list(WS_RANGE), 5):
    for ti, ct in random.sample(sorted(itertools.product(TIs, CTs)), 200):
        # print(f"{wind_speed=}\t{ti=}\t{ct=}")
        site = py_wake_utils.get_site(ti=ti, ws=wind_speed)
        wind_turbine = py_wake_utils.get_wind_turbine(
            TURBINE_DIAMETER,
            TURBINE_HUB_HEIGHT,
            TURBINE_POWER_NORM,
            constant_ct=ct,
            ti=ti,
        )

        # single wake model
        ainslie_model = EddyViscosityModel(site, wind_turbine)

        df = (
            py_wake_utils.generate_wake_dataset(
                ainslie_model,
                wind_speed,
                WIND_DIRECTION,
                TURBINE_DIAMETER,
                TURBINE_X,
                TURBINE_Y,
                horizontal_grid,
                wind_turbine,
            )
            .to_dataframe()
            .reset_index()
            .rename(columns={"x:D": "x/D", "y:D": "y/D"})
            .sort_values(by=["x/D", "y/D"])
        )

        filtered_df = df[df["wind_deficit"] > DEFICIT_THRESHOLD]
        if filtered_df.empty:
            utils.plotting.plot_deficit_map(df)
            continue

        x_range = np.nanmin(filtered_df["x/D"]), np.nanmax(filtered_df["x/D"])
        y_range = np.nanmin(filtered_df["y/D"]), np.nanmax(filtered_df["y/D"])

        # print("Range of x/D:", x_range)
        # print("Range of y/D:", y_range)
        x_ranges.append(x_range)
        y_ranges.append(y_range)

In [ ]:
import statistics

x_ends = sorted([x_range[1] for x_range in x_ranges])
print(
    f"For x-end, min is: {min(x_ends)} and max is: {max(x_ends)},"
    + f" with an average of {sum(x_ends) / len(x_ends)} and a median of {statistics.median(x_ends)}"
)

y_starts = sorted([y_range[0] for y_range in y_ranges])
print(
    f"For y-start, min is: {min(y_starts)} and max is: {max(y_starts)},"
    + f" with an average of {sum(y_starts) / len(y_starts)} and a median of {statistics.median(y_starts)}"
)
y_ends = sorted([y_range[1] for y_range in y_ranges])
print(
    f"For y-end, min is: {min(y_ends)} and max is: {max(y_ends)},"
    + f" with an average of {sum(y_ends) / len(y_ends)} and a median of {statistics.median(y_ends)}"
)

# Analysis of Ainslie-generated dataset

### Correlation matrix

In [ ]:
folder = "data/discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01"
total_data = list()
for file in os.listdir(folder):
    if file.startswith("."):
        continue
    wind_speed = int(file.split(".nc")[0].split("ws_")[1])
    df = load_netcfd(
        folder,
        wind_speed,
        include_ws_column=True,
        input_var_to_reduction_factor={
            "ti": 4,
            "ct": 4,
        },
    )
    df.drop("WS_eff", axis=1, inplace=True)
    df.rename({"ti": "TI", "ct": "C_T", "wind_deficit": "wd"}, axis=1, inplace=True)
    total_data.append(df)

total_data = pd.concat(total_data)

# Calculate the correlation matrix
correlation_matrix = total_data.corr()

# Plot a heatmap to visualize the correlation
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
# plt.title("Correlation Matrix of Input Features and Wind Deficit")
plt.show()

In [ ]:
"""
folder = "data/discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01"
aggregated_data = pd.DataFrame()

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    wind_speed = int(file.split(".nc")[0].split("ws_")[1])
    df = load_netcfd(folder, wind_speed, include_ws_column=True)
    
    # Compute the desired statistics for each file and append them to the aggregated DataFrame
    statistics = df.mean()  # Replace this with your desired statistics calculation
    aggregated_data = aggregated_data.append(statistics, ignore_index=True)

# Calculate the correlation matrix for the aggregated data
correlation_matrix = aggregated_data.corr()

# Plot a heatmap to visualize the correlation
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Matrix")
plt.show()
"""

## Deficit statistics

In [ ]:
# Calculation
folder = "data/discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01"

ws_to_metrics = dict()
tict_to_ws_to_deficitmetrics = dict()
deficits = list()
for file in os.listdir(folder):
    if file.startswith("."):
        continue
    wind_speed = int(file.split(".nc")[0].split("ws_")[1])
    df = load_netcfd(
        folder, wind_speed, include_ws_column=True
    )  # , ti_range=(0, 0.5), ct_range=(0, 0.5),
    # input_var_to_reduction_factor={'ti': 3, 'ct': 5})

    deficitmean = df.wind_deficit.mean()
    deficitmin = df.wind_deficit.min()
    deficitmax = df.wind_deficit.max()
    deficitmedian = df.wind_deficit.median()
    ws_to_metrics[wind_speed] = [deficitmin, deficitmean, deficitmedian, deficitmax]

    print(deficitmin, deficitmax, deficitmean, deficitmedian)
    plt.boxplot(df.wind_deficit)
    plt.xlabel("wind deficit")
    plt.ylabel("Value")
    plt.ylim(0, 0.2)
    plt.show()

    sns.violinplot(y=df.wind_deficit)
    plt.ylabel("wind deficit")
    plt.ylim(0, 0.2)
    plt.show()
    break

    """
    for inputs, subdf in df.groupby(["ti", "ct", "ws"]):
        #assert subdf.shape[0] == 7168
        tict = "{:.2f}_{:.2f}".format(inputs[0], inputs[1])
        ws = inputs[2]
        deficitmean = subdf.wind_deficit.mean()
        deficitmin = subdf.wind_deficit.min()
        deficitmax = subdf.wind_deficit.max()
        deficitmedian = subdf.wind_deficit.median()
        #print(tict, ws, deficitmean)
        if tict not in tict_to_ws_to_deficitmetrics.keys():
            tict_to_ws_to_deficitmetrics[tict] = dict()
        tict_to_ws_to_deficitmetrics[tict][ws] = [deficitmin, deficitmean, deficitmedian, deficitmax]
    """

print(len(tict_to_ws_to_deficitmetrics))

In [ ]:
deficits = np.array(deficits)
print(deficits.min(), deficits.max(), deficits.mean())

### Overall Deficit statistics over wind speed

In [ ]:
# find the maximum deficit value
for ws, metrics in ws_to_metrics.items():
    print(
        f"for {ws=} \t-> mean={metrics[0]}, median={metrics[2]}, min={metrics[1]}, max={metrics[-1]}"
    )

In [ ]:
sorted_data = sorted(ws_to_metrics.items())

# Extract sorted keys and values
x = [key for key, _ in sorted_data]
ys = [[value[i] for _, value in sorted_data] for i in range(4)]

colors = ["red", "grey", "blue", "green"]
labels = ["min", "mean", "median", "max"]
for i, y in enumerate(ys):
    plt.plot(x, y, marker="^", color=colors[i], label=labels[i])
plt.xlabel("Wind speed (m/s)")
plt.ylabel("Wind deficit")
plt.legend()
plt.show()

### Deficit statistics over wind speed per TI-CT combination

In [ ]:
selected_ticts = random.sample(tict_to_ws_to_deficitmetrics.items(), 5)

for tict, ws_to_deficitmetrics in selected_ticts:
    print(tict)
    sorted_data = sorted(ws_to_deficitmetrics.items())

    # Extract sorted keys and values
    x = [key for key, _ in sorted_data]
    ys = [[value[i] for _, value in sorted_data] for i in range(4)]

    colors = ["red", "grey", "blue", "green"]
    labels = ["min", "mean", "median", "max"]
    for i, y in enumerate(ys):
        plt.plot(x, y, marker="^", color=colors[i], label=labels[i])
    plt.xlabel("Wind speed (m/s)")
    plt.ylabel("Wind deficit")
    plt.legend()
    plt.show()

## Feature importance (from Decision Tree results)

In [ ]:
feature_to_importance = {
    "TI": 0.21651978373434827,
    "C_T": 0.20347379365589874,
    "ws": 2.269498858710338e-18,
    "x/D": 0.14726651526616422,
    "y/D": 0.4327399073435887,
}

colors = ["skyblue", "lightgreen", "salmon", "lightcoral", "lightpink"]

plt.figure(figsize=(8, 6))
plt.bar(feature_to_importance.keys(), feature_to_importance.values(), color=colors)

plt.xlabel("Features")
plt.ylabel("Feature Importance")
# plt.title('Feature Importance of Decision Tree Regressor')
plt.show()

# Time comparison between pywake and my model simulations

In [ ]:
# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)

# default parameters
TURBINE_X = [0]
TURBINE_Y = [0]
WIND_DIRECTION = 270

# IEA37 values
TURBINE_DIAMETER = 198
TURBINE_HUB_HEIGHT = 119
TURBINE_POWER_NORM = 10000

# discretization factors
X_START_FACTOR = 2
X_END_FACTOR = 20
Y_START_FACTOR = -5
Y_END_FACTOR = 5
GRID_STEP_FACTOR = 1 / 8

# parameters for data generation
WS_RANGE = range(3, 26)
TI_STEP = 0.01
CT_STEP = 0.01
TIs = utils.my_arange(0, 1, TI_STEP)
CTs = utils.my_arange(0.2, 24 / 25, CT_STEP)

In [ ]:
horizontal_grid = py_wake_utils.get_discretized_grid(
    TURBINE_DIAMETER,
    X_START_FACTOR,
    X_END_FACTOR,
    Y_START_FACTOR,
    Y_END_FACTOR,
    GRID_STEP_FACTOR,
)

ws_to_list = dict()
for wind_speed in WS_RANGE:
    datasets = list()

    start_time = time.time()

    for ti, ct in itertools.product(TIs, CTs):
        site = py_wake_utils.get_site(ti=ti, ws=wind_speed)
        wind_turbine = py_wake_utils.get_wind_turbine(
            TURBINE_DIAMETER,
            TURBINE_HUB_HEIGHT,
            TURBINE_POWER_NORM,
            constant_ct=ct,
            ti=ti,
        )

        # single wake model
        ainslie_model = EddyViscosityModel(site, wind_turbine)

        ds = py_wake_utils.generate_wake_dataset(
            ainslie_model,
            wind_speed,
            WIND_DIRECTION,
            TURBINE_DIAMETER,
            TURBINE_X,
            TURBINE_Y,
            horizontal_grid,
            wind_turbine,
        )
        datasets.append(ds)

    filepath = data_utils.get_filepath(
        X_START_FACTOR,
        X_END_FACTOR,
        Y_START_FACTOR,
        Y_END_FACTOR,
        GRID_STEP_FACTOR,
        wind_speed,
        TI_STEP,
        CT_STEP,
    )
    final_ds = xr.concat(
        [d.stack(z=["x:D", "y:D", "ti", "ct"]) for d in datasets], "z"
    ).unstack("z")
    cacca = final_ds.load()

    end_time = time.time()
    execution_time = end_time - start_time
    num_simulations = len(datasets)
    print(
        f"For {wind_speed=}] Execution time: {execution_time} seconds for {num_simulations} simulations -> {(execution_time/num_simulations):2f}s per simulation"
    )
    ws_to_list[wind_speed] = [num_simulations, execution_time]

tot_simulations = sum([num_sim for num_sim, _ in ws_to_list.values()])
tot_time = sum([ex_time for _, ex_time in ws_to_list.values()])
print(
    f"\nOverall execution time for {tot_simulations} simulations: {tot_time} seconds -> {tot_time/tot_simulations:2f}s per simulation"
)

In [ ]:
MODEL_PATH = "saved_models/discr_factors_x2_20_y-5_5_step0.125_TIstep0.01_CTstep0.01/multivariate_NN_layers50-500-2500.pt"
HIDDEN_LAYERS_UNITS = [
    int(lu) for lu in MODEL_PATH.split("layers")[-1].split(".pt")[0].split("-")
]


class MultivariateNN(nn.Module):
    def __init__(self, input_space, output_space) -> None:
        super(MultivariateNN, self).__init__()
        layer_units = [input_space] + HIDDEN_LAYERS_UNITS + [output_space]
        layers = []
        for first, second in zip(layer_units, layer_units[1:]):
            layers += [nn.Linear(first, second), nn.ReLU()]
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


model = MultivariateNN(2, 11520)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

ws_to_list = dict()
with torch.no_grad():
    for wind_speed in WS_RANGE:
        datasets = []

        start_time = time.time()

        for ti, ct in itertools.product(TIs, CTs):
            wake_field = model(torch.FloatTensor([ti, ct]))
            datasets.append(wake_field)

        end_time = time.time()
        execution_time = end_time - start_time
        num_simulations = len(datasets)
        print(
            f"For {wind_speed=}] Execution time: {execution_time} seconds for {num_simulations} simulations -> {(execution_time/num_simulations):2f}s per simulation"
        )
        ws_to_list[wind_speed] = [num_simulations, execution_time]

tot_simulations = sum(num_sim for num_sim, _ in ws_to_list.values())
tot_time = sum(ex_time for _, ex_time in ws_to_list.values())
print(
    f"\nOverall execution time for {tot_simulations} simulations: {tot_time} seconds -> {tot_time/tot_simulations:2f}s per simulation"
)

# Analysis of "Evaluation of 3 potential ML algorithms" dataset

In [ ]:
data = pd.read_csv("data/Evaluation of three potential ML algorithms - train-test.csv")
data

In [ ]:
Xs = data["s (x/D)"].unique()
Ys = data["r"].unique()
shape = len(Xs), len(Ys)
shape

In [ ]:
X_LABEL = "s (x/D)"
Y_LABEL = "r"  # or 'r/R
Z_LABEL = "Velocity"
GROUPBY = "Wind speed"

In [ ]:
def plot_contour(
    X,
    Y,
    Z,
    xlabel: str,
    ylabel: str,
    zlabel: str,
    title: str,
    levels=500,
    cmap: str = "Blues",
    ax=None,
) -> None:
    ax = plt.gca()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    c = ax.contourf(X, Y, Z, levels=levels, cmap=cmap)
    plt.colorbar(c, label=zlabel, ax=ax)
    plt.show()


for wind_speed, subdf in data.groupby(GROUPBY):
    assert not subdf[[X_LABEL, Y_LABEL]].duplicated().any()
    X, Y = np.meshgrid(subdf[X_LABEL].unique(), subdf[Y_LABEL].unique())
    Z = subdf.pivot(index=Y_LABEL, columns=X_LABEL, values="Velocity").values
    plot_contour(X, Y, Z, X_LABEL, Y_LABEL, Z_LABEL, title=f"Wind speed: {wind_speed}")

In [ ]:
for wind_speed, subdf in data.groupby(GROUPBY):
    s_values = subdf[X_LABEL]
    r_values = subdf[Y_LABEL]
    velocity_values = subdf[Z_LABEL]

    X, Y = np.meshgrid(s_values.unique(), r_values.unique())

    # Create an empty 2D array for counter values
    counter_values = np.zeros_like(X)

    # Fill the counter values array with corresponding velocity values
    for s, r, velocity in zip(s_values, r_values, velocity_values):
        s_index = np.where(X[0] == s)[0][0]
        r_index = np.where(Y[:, 0] == r)[0][0]
        counter_values[r_index, s_index] = velocity

    plot_contour(
        X,
        Y,
        counter_values,
        X_LABEL,
        Y_LABEL,
        Z_LABEL,
        title=f"Counter Map - Wind Speed: {wind_speed}",
    )